In [11]:
import sys
!conda install --yes --prefix {sys.prefix} keras

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==2018.12=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::numba==0.41.0=py37h962f231_0
done

## Package Plan ##

  environment location: /opt/ds-data/home/aaron.hogancamp/anaconda3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB
    anaconda-custom            |           py37_1           3 KB
    soupsieve-1.9.2            |           py37_0          61 KB
    tbb-2019.4                 |       hfd86e86_0         1.4 MB
    zipp-0.5.2                 |             py_0           8 KB
    ------------------------------------------------------------
                                    

In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

path = "/opt/ds-data/home/aaron.hogancamp/CatsVsDogs/data"
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img(path + '/train/cats/cat000.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=path+'/preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely
        


In [5]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = path + '/train'
validation_data_dir = path + '/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
print("input shape defined")
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
print("data generator instantiated")
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
print("images scaled")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
#    batch_size=batch_size,
    class_mode='binary')

print("data loaded")
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
#    batch_size=batch_size,
    class_mode='binary')

print("batch size is: " + str(batch_size))
print("fitting model")
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
#    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=100)
    #,
    #validation_steps=nb_validation_samples // batch_size)
    

print("model fit complete")
model.save_weights('first_try.h5')

input shape defined
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
data generator instantiated
images scaled
Found 2050 images belonging to 3 classes.
data loaded
Found 825 images belonging to 3 classes.
batch size is: 16
fitting model
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
100/100 [==============================] - 35s 353ms/step - loss: 0.6942 - acc: 0.4919 - val_loss: 0.6453 - val_acc: 0.4888
Epoch 2/50
100/100 [==============================] - 33s 328ms/step - loss: 0.6369 - acc: 0.5904 - val_loss: 0.5905 - val_acc: 0.6138
Epoch 3/50
100/100 [==============================] - 33s 333ms/step - loss: 0.5905 - acc: 0.6478 - val_loss: 0.5771 - val_acc: 0.6510
Epoch 4/50
100/100 [==============================] - 33s 329ms/step - loss: 0.5665 - acc: 0.6638 - val_loss: 0.5608 - val_acc: 0.6611
Epoch 5/50
100/100 [================

In [7]:
print(path+'/preview')

/opt/ds-data/home/aaron.hogancamp/CatsVsDogs/data/preview


In [ ]:
#from shutil import copyfile

#path = "/Users/timkucejko/Documents/Jupyter notebooks/Experimental/CatsVsDogs/data"

#for i in range(1000):
    
#   copyfile(path + '/train2/cat.' + str(i) + '.jpg', path + '/train/cats/cat' + str(i).zfill(3) + '.jpg')
#    copyfile(path + '/train2/dog.' + str(i) + '.jpg', path + '/train/dogs/dog' + str(i).zfill(3) + '.jpg')

#from shutil import copyfile

#path = "/Users/timkucejko/Documents/Jupyter notebooks/Experimental/CatsVsDogs/data"

#for i in range(1000, 1400):
    
#    copyfile(path + '/train2/cat.' + str(i) + '.jpg', path + '/validation/cats/cat' + str(i).zfill(3) + '.jpg')
#    copyfile(path + '/train2/dog.' + str(i) + '.jpg', path + '/validation/dogs/dog' + str(i).zfill(3) + '.jpg')


In [8]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

print("defining functions")
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
#        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("predicting training features")
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    print("generating training predictions")
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    print("saved bottleneck features train")
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
 #       batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    print("generating validation predictions")
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    print("saved bottleneck features validation")

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    print('str[0]: ' + str([0]))
    print('str[1]: ' + str([1]))
    print('nb_validation_samples: ' + str(nb_validation_samples))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    print("loaded training features")
    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    print("loaded validation features")
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.65))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    print("fitting model")
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    print("saving weights")
    model.save_weights(top_model_weights_path)


#save_bottlebeck_features()
train_top_model()

defining functions
str[0]: [0]
str[1]: [1]
nb_validation_samples: 800
loaded training features
loaded validation features
fitting model
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.7171 - acc: 0.7395 - val_loss: 0.2842 - val_acc: 0.8850
Epoch 2/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.4119 - acc: 0.8310 - val_loss: 0.4699 - val_acc: 0.7987
Epoch 3/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3397 - acc: 0.8640 - val_loss: 0.2459 - val_acc: 0.9062
Epoch 4/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3199 - acc: 0.8810 - val_loss: 0.3277 - val_acc: 0.8688
Epoch 5/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2749 - acc: 0.8890 - val_loss: 0.2787 - val_acc: 0.8988
Epoch 6/50
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2416 - acc: 0.9125 - val_loss: 0.5688 - val_acc: 0.8200
Epoch 7/50
20

In [9]:
import keras as k;
print(k.__version__)

2.2.4


In [10]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import backend as K
K.set_image_data_format('channels_last')

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'cats_and_dogs_small/train'
validation_data_dir = 'cats_and_dogs_small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16
#model = applications.VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))
# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
print(str(model.output_shape))

top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

58892288/58889256 [==============================] - 13s 0us/step
Model loaded.
(None, None, None, 512)


ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 512). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.